In [469]:
import sympy
import numpy as np
import pandas as pd

### Daten einlesen

In [470]:
with open("../Daten/chemischesSystem1.txt", "r") as file:
    data = [line.rstrip() for line in file]


def get_spec_ele(data: list[str]) -> tuple[list]:
    ns = data[0].split(";")
    ne = data[-1].split(";")
    return ns, ne

specs, ele = get_spec_ele(data)
ns = len(specs)
ne = len(ele)

In [471]:
print(specs)
print(ns)

print(ele)
print(ne)

['N,2', 'O,2', 'H,2,O,1', 'N,1,O,1', 'N,1,O,2', 'N,1,H,3', 'H,1,N,1,O,3']
7
['H', 'N', 'O']
3


### Formel Matrix **A**


In [472]:
def get_formel_matrix_A(specs: list[str], eles: list[str]):
    counts_list = []

    for s in specs:
        counts = {}
        tokens = s.split(",")
        for i in range(0, len(tokens), 2):
            element = tokens[i]
            count = int(tokens[i+1])
            counts[element] = count
        counts_list.append(counts)

    result = []
    for element in eles:
        counts_for_element = []
        for counts in counts_list:
            counts_for_element.append(counts.get(element, 0))
        result.append(counts_for_element)


    return np.array(result)

A = get_formel_matrix_A(specs, ele)
A

array([[0, 0, 2, 0, 0, 3, 1],
       [2, 0, 0, 1, 1, 1, 1],
       [0, 2, 1, 1, 2, 0, 3]])

### Gauß Algorithmus

In [473]:
A = sympy.Matrix(A)
matrix, dim = A.rref()

### A^

In [474]:
start_index = dim[-1] + 1
cols = [col for col in range(start_index, matrix.cols)]
A_hut = matrix[:, cols] * -1
A_hut



Matrix([
[-1/2, -1/2, -1/2, -1/2],
[-1/2,   -1,  3/4, -5/4],
[   0,    0, -3/2, -1/2]])

### Y

In [475]:
Y = A_hut.row_insert(A_hut.rows, sympy.Matrix.eye(A_hut.cols))
Y

Matrix([
[-1/2, -1/2, -1/2, -1/2],
[-1/2,   -1,  3/4, -5/4],
[   0,    0, -3/2, -1/2],
[   1,    0,    0,    0],
[   0,    1,    0,    0],
[   0,    0,    1,    0],
[   0,    0,    0,    1]])

### Reaktions Tabelle

In [476]:
data = {"Chemical Species": specs}

for i in range(Y.shape[1]):
    data[f"Reaction {i+1}"] = [Y[j, i] for j in range(Y.shape[0])]

df = pd.DataFrame(data)

df

,Chemical Species,Reaction 1,Reaction 2,Reaction 3,Reaction 4
0,"N,2",-1/2,-1/2,-1/2,-1/2
1,"O,2",-1/2,-1,3/4,-5/4
2,"H,2,O,1",0,0,-3/2,-1/2
3,"N,1,O,1",1,0,0,0
4,"N,1,O,2",0,1,0,0
5,"N,1,H,3",0,0,1,0
6,"H,1,N,1,O,3",0,0,0,1
